In [1]:
import os
import cv2
import numpy as np
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, Conv2DTranspose, Conv2D
from keras.preprocessing.image import ImageDataGenerator
import keras
from keras.applications.vgg16 import VGG16

class gen_args:
    data_dir = None
    data_ext = None
    def __init__(self,dirr,ext):
        self.data_dir = dirr
        self.data_ext = ext
        
    
def random_crop(image, crop_size):
    cropy, cropx = crop_size
    height, width = image.shape[:-1]
    cy = cropy - height
    if cy > 0:
    	if cy % 2 == 0:
    		image = np.vstack((np.zeros((cy/2,width,3)) , image , np.zeros((cy/2,width,3))))
    		#print image.shape
    	else:
    		image = np.vstack((np.zeros((cy/2,width,3)) , image , np.zeros((cy/2 +1,width,3))))
    		#print image.shape
    if cy < 0:
    	if cy % 2 == 0:
    		image = np.delete(image, range(-1*cy/2), axis = 0)
    		#print image.shape
    		image = np.delete(image, range(height + cy,height +  cy/2), axis = 0)
    		#print image.shape
    	else:
    		image = np.delete(image, range(-1*cy/2), axis =0)
    		image = np.delete(image, range(height + cy, height + cy/2 + 1), axis=0)
    #print image.shape
    height, width = image.shape[:-1]
    cx = cropx - width
    if cx > 0:
        if cx % 2 == 0:
    		image = np.hstack((np.zeros((height,cx/2,3)) , image , np.zeros((height,cx/2,3))))
    	else:
    		image = np.hstack((np.zeros((height,cx/2,3)) , image , np.zeros((height,cx/2 + 1,3))))
    if cx < 0:
    	if cx % 2 == 0:
    		image = np.delete(image, range(-1*cx/2), axis = 1)
    		image = np.delete(image, range(width + cx,width +  cx/2), axis = 1)
    	else:
    		image = np.delete(image, range(-1*cx/2), axis =1)
    		image = np.delete(image, range(width + cx, width + cx/2 + 1), axis=1)
    #print image.shape
    return image


def Segment_datagen(file_path, rgb_args, nir_args, label_args, batch_size, input_size):
    # Create MEMORY enough for one batch of input(s) & labels
    data = np.zeros((2,batch_size,input_size[0],input_size[1],3))
    labels = np.zeros((batch_size,input_size[0],input_size[1],3))
    # Read the file names
    files = open(file_path)
    names = files.readlines()
    files.close()
    # Enter the indefinite loop of generator
    while True:
        random_samples = np.random.randint(len(names), size=batch_size)
        for i in range(batch_size):
            #index_of_random_sample = np.random.choice(len(names))
            data[0][i] = random_crop(cv2.imread(rgb_args.data_dir+names[random_samples[i]].strip('\n')+rgb_args.data_ext), input_size)
            data[1][i]= random_crop(cv2.imread(nir_args.data_dir+names[random_samples[i]].strip('\n')+nir_args.data_ext), input_size)
            labels[i] = random_crop(cv2.imread(label_args.data_dir+names[random_samples[i]].strip('\n')+label_args.data_ext), input_size)
        yield [data[0],data[1]],labels

RGB_args = gen_args ('/home/captain_jack/Downloads/freiburg_forest_annotated/train/rgb/','.jpg')
NIR_args = gen_args ('/home/captain_jack/Downloads/freiburg_forest_annotated/train/nir_color/','.png')
Label_args = gen_args ('/home/captain_jack/Downloads/freiburg_forest_annotated/train/GT_color/','.png')

generator = Segment_datagen(
    file_path = '/home/captain_jack/Downloads/freiburg_forest_annotated/train/rgb/train.txt',
    rgb_args = RGB_args,
    nir_args = NIR_args,
    label_args = Label_args,
    batch_size= 8,
    input_size=[512,896])

vgg16_conv = VGG16(weights='imagenet', include_top=False)
# RGB PART OF CNN
inputs_rgb = Input(shape=(512,896,3))
rgb_vgg_out = vgg16_conv(inputs_rgb)
conv_rgb1 = Conv2D(8, (3,3), strides=(1, 1), padding = 'same', activation='relu',data_format="channels_last") (rgb_vgg_out)
#conv_rgb2 = Conv2D(16, (3,3), strides=(1, 1), padding = 'same', activation='relu') (conv_rgb1)
pool_rgb1 = MaxPooling2D(pool_size=(2, 2), strides=2, padding='same') (conv_rgb1)
'''conv_rgb3 = Conv2D(32, (3,3), strides=(1, 1), padding = 'same', activation='relu') (pool_rgb1)
conv_rgb4 = Conv2D(64, (3,3), strides=(1, 1), padding = 'same', activation='relu') (conv_rgb3)
pool_rgb2 = MaxPooling2D(pool_size=(2, 2), strides=2, padding='same') (conv_rgb4)
conv_rgb5 = Conv2D(128, (3,3), strides=(1, 1), padding = 'same', activation='relu') (pool_rgb2)
conv_rgb6 = Conv2D(512, (3,3), strides=(1, 1), padding = 'same', activation='relu') (conv_rgb5)
pool_rgb3 = MaxPooling2D(pool_size=(2, 2), strides=2, padding='same') (conv_rgb6)'''
# NIR PART OF CNN
inputs_nir = Input(shape=(512,896,3))
nir_vvg_out = vgg16_conv(inputs_nir)
conv_nir1 = Conv2D(8, (3,3), strides=(1, 1), padding = 'same', activation='relu',data_format="channels_last") (nir_vvg_out)
#conv_nir2 = Conv2D(16, (3,3), strides=(1, 1), padding = 'same', activation='relu') (conv_nir1)
pool_nir1 = MaxPooling2D(pool_size=(2, 2), strides=2, padding='same') (conv_nir1)
'''conv_nir3 = Conv2D(32, (3,3), strides=(1, 1), padding = 'same', activation='relu') (pool_nir1)
conv_nir4 = Conv2D(64, (3,3), strides=(1, 1), padding = 'same', activation='relu') (conv_nir3)
pool_nir2 = MaxPooling2D(pool_size=(2, 2), strides=2, padding='same') (conv_nir4)
conv_nir5 = Conv2D(128, (3,3), strides=(1, 1), padding = 'same', activation='relu') (pool_nir2)
conv_nir6 = Conv2D(512, (3,3), strides=(1, 1), padding = 'same', activation='relu') (conv_nir5)
pool_nir3 = MaxPooling2D(pool_size=(2, 2), strides=2, padding='same') (conv_nir6)'''
# CONACTENATE the ends of RGB & NIR 
k = keras.layers.concatenate([pool_nir1, pool_rgb1])
# Deconvolution Layers
deconv1 = Conv2DTranspose(6, (4,4), strides=(2, 2), padding='same', data_format="channels_last", activation='relu') (k)
deconv2 = Conv2DTranspose(6, (4,4), strides=(2, 2), padding='same', activation='relu') (deconv1)
deconv3 = Conv2DTranspose(3, (4,4), strides=(2, 2), padding='same', activation='relu') (deconv2)
deconv4 = Conv2DTranspose(3, (4,4), strides=(2, 2), padding='same', activation='relu') (deconv3)
deconv5 = Conv2DTranspose(3, (4,4), strides=(2, 2), padding='same', activation='relu') (deconv4)
deconv6 = Conv2DTranspose(3, (4,4), strides=(2, 2), padding='same', activation='relu') (deconv5)

# INPUTS & OUTPUTS
model = Model(inputs=[inputs_rgb,inputs_nir], outputs=[deconv6])
print 'compiling'
model.compile(optimizer='rmsprop',
              loss='hinge',
              metrics=['accuracy'])
model.summary()
#model.fit_generator(generator,steps_per_epoch=200,epochs=10)


Using TensorFlow backend.


compiling
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_3 (InputLayer)             (None, 512, 896, 3)   0                                            
____________________________________________________________________________________________________
input_2 (InputLayer)             (None, 512, 896, 3)   0                                            
____________________________________________________________________________________________________
vgg16 (Model)                    multiple              14714688    input_2[0][0]                    
                                                                   input_3[0][0]                    
____________________________________________________________________________________________________
conv2d_2 (Conv2D)                (None, 16, 28, 8)     36872       vgg16[2][0]   

In [1]:
# Unimodal image generator for segmentation
def Segment_datagen(file_path,data_dir,data_ext,label_dir,label_ext,batch_size,input_size):
    print "Entered Segment_datagen"
    data = np.zeros((batch_size,input_size[0],input_size[1],3))
    labels = np.zeros((batch_size,input_size[0],input_size[1],3))
    files = open(file_path)
    names = files.readlines()
    files.close()
    print len(names)
    while True:
        for i in range(batch_size):
            print 'batch_no'+str(i)
            index_of_random_sample = np.random.choice(len(names))
            data[i] = random_crop(cv2.imread(data_dir+names[index_of_random_sample].strip('\n')+data_ext), input_size)
            print 'read'
            labels[i] = random_crop(cv2.imread(label_dir+names[index_of_random_sample].strip('\n')+label_ext), input_size)
        yield data,labels
        
        

In [37]:
class gen_args:
    data_dir = None
    data_ext = None
    def __init__(self,dirr,ext):
        self.data_dir = dirr
        self.data_ext = ext

        
RGB_args = gen_args ('/home/captain_jack/Downloads/freiburg_forest_annotated/train/rgb/','.jpg')
NIR_args = gen_args ('/home/captain_jack/Downloads/freiburg_forest_annotated/train/nir_color/','.png')
label_args = gen_args ('/home/captain_jack/Downloads/freiburg_forest_annotated/train/GT_color/','.png')
file_path = '/home/captain_jack/Downloads/freiburg_forest_annotated/train/rgb/train.txt'
rgb_args = RGB_args
nir_args = NIR_args
label_args = Label_args
batch_size= 8
input_size=[512,904]

# Create MEMORY enough for one batch of input(s) & labels
data = np.zeros((2,batch_size,input_size[0],input_size[1],3))
labels = np.zeros((batch_size,input_size[0],input_size[1],3))
# Read the file names
files = open(file_path)
names = files.readlines()
files.close()
# Enter the indefinite loop of generator
for i in range(batch_size):
    index_of_random_sample = np.random.choice(len(names))
    data[0][i] = random_crop(cv2.imread(rgb_args.data_dir+names[index_of_random_sample].strip('\n')+rgb_args.data_ext), input_size)
    data[1][i] = random_crop(cv2.imread(nir_args.data_dir+names[index_of_random_sample].strip('\n')+nir_args.data_ext), input_size)
    labels[i] = random_crop(cv2.imread(label_args.data_dir+names[index_of_random_sample].strip('\n')+label_args.data_ext), input_size)
print data,labels
    
data.shape
data[:][0].shape

[[[[[ 0.  0.  0.]
    [ 0.  0.  0.]
    [ 0.  0.  0.]
    ..., 
    [ 0.  0.  0.]
    [ 0.  0.  0.]
    [ 0.  0.  0.]]

   [[ 0.  0.  0.]
    [ 0.  0.  0.]
    [ 0.  0.  0.]
    ..., 
    [ 0.  0.  0.]
    [ 0.  0.  0.]
    [ 0.  0.  0.]]

   [[ 0.  0.  0.]
    [ 0.  0.  0.]
    [ 0.  0.  0.]
    ..., 
    [ 0.  0.  0.]
    [ 0.  0.  0.]
    [ 0.  0.  0.]]

   ..., 
   [[ 0.  0.  0.]
    [ 0.  0.  0.]
    [ 0.  0.  0.]
    ..., 
    [ 0.  0.  0.]
    [ 0.  0.  0.]
    [ 0.  0.  0.]]

   [[ 0.  0.  0.]
    [ 0.  0.  0.]
    [ 0.  0.  0.]
    ..., 
    [ 0.  0.  0.]
    [ 0.  0.  0.]
    [ 0.  0.  0.]]

   [[ 0.  0.  0.]
    [ 0.  0.  0.]
    [ 0.  0.  0.]
    ..., 
    [ 0.  0.  0.]
    [ 0.  0.  0.]
    [ 0.  0.  0.]]]


  [[[ 0.  0.  0.]
    [ 0.  0.  0.]
    [ 0.  0.  0.]
    ..., 
    [ 0.  0.  0.]
    [ 0.  0.  0.]
    [ 0.  0.  0.]]

   [[ 0.  0.  0.]
    [ 0.  0.  0.]
    [ 0.  0.  0.]
    ..., 
    [ 0.  0.  0.]
    [ 0.  0.  0.]
    [ 0.  0.  0.]]

   [[ 0.  0.  0.]
    [ 0.  0

(8, 512, 904, 3)

In [12]:
import numpy as np
import cv2
batch_size = 8

files = open('/home/captain_jack/Downloads/freiburg_forest_annotated/train/rgb/train.txt')
names = files.readlines()
files.close()

RGB_args = gen_args ('/home/captain_jack/Downloads/freiburg_forest_annotated/train/rgb/','.jpg')
NIR_args = gen_args ('/home/captain_jack/Downloads/freiburg_forest_annotated/train/nir_color/','.png')
Label_args = gen_args ('/home/captain_jack/Downloads/freiburg_forest_annotated/train/GT_color/','.png')
random_samples = np.random.randint(len(names), size=batch_size)
rgb_args = RGB_args
nir_args = NIR_args
label_args = Label_args
input_size=[512,904]

data = np.zeros((2,batch_size,input_size[0],input_size[1],3))
labels = np.zeros((batch_size,input_size[0],input_size[1],3))
for i in range(batch_size):
    print i
    #index_of_random_sample = np.random.choice(len(names))
    data[0][i] = random_crop(cv2.imread(rgb_args.data_dir+names[random_samples[i]].strip('\n')+rgb_args.data_ext), input_size)
    data[1][i]= random_crop(cv2.imread(nir_args.data_dir+names[random_samples[i]].strip('\n')+nir_args.data_ext), input_size)
    labels[i] = random_crop(cv2.imread(label_args.data_dir+names[random_samples[i]].strip('\n')+label_args.data_ext), input_size)
print [data[0],data[1]],labels


0
1
2
3
4
5
6
7
[array([[[[ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         ..., 
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.]],

        [[ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         ..., 
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.]],

        [[ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         ..., 
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.]],

        ..., 
        [[ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         ..., 
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.]],

        [[ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         ..., 
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.]],

        [[ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         ..., 
         [ 

In [18]:
from keras.applications.vgg16 import VGG16


inputs_rgb = Input(shape=(512,904,3))
rgb_model_vgg16_conv = VGG16(weights='imagenet', include_top=False)
rgb_vgg_out = rgb_model_vgg16_conv(inputs_rgb)
conv_rgb1 = Conv2D(8, (3,3), strides=(1, 1), padding = 'same', activation='relu',data_format="channels_last") (vgg_out)

NameError: name 'vgg_out' is not defined